<a href="https://colab.research.google.com/github/miyuu157/LB_miyuu/blob/main/Fashion_1day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 課題

CNNを用いて、FashionMNISTの高精度な分類器を実装してみましょう。</br>
モデルのレイヤーを変更してみるなどして精度の向上にチャンレンジして下さい。

### 目標値

Accuracy: 93%

### ルール

- 訓練データはx_train、 t_train、テストデータはx_test、t_testで与えられます。ご自身のマイドライブにコピーして作業を始めてください。
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください。
- **下のセルで指定されているx_train、t_train以外の学習データは使わないでください。**
- torchvision等で既に実装されているモデルを使用しないで下さい。**ただし、torchvision.transformsのみ使用しても良いです。**
- Conv, Pool, Flatten, Denseは必ず下記のクラスを補完する形で作成してください．（nn.Conv2D, nn.Pool2d, nn.Flatten, nn.Dense等を使用しないでください．）
- n_epoch <= 20 （学習時間が伸び1dayに収まらない可能性があります）
- input size <= 32x32



### 評価方法

- 予測ラベルのx_testに対する精度 (Accuracy) で評価します。


### データの読み込み　※この部分は変更しないでください

In [3]:
# この部分は変更しないでください
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# この部分は変更しないでください
import gzip
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
import inspect

from torchvision import transforms
from PIL import Image

#学習データ

with gzip.open('/content/drive/MyDrive/data/train-labels-idx1-ubyte.gz', 'rb') as lbpath:
    t_train = np.frombuffer(lbpath.read(), dtype=np.uint8, offset=8)
with gzip.open('/content/drive/MyDrive/data/train-images-idx3-ubyte.gz', 'rb') as imgpath:
    x_train = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(len(t_train), 784)

val_size = 10000 

#テストデータ
with gzip.open('/content/drive/MyDrive/data/t10k-labels-idx1-ubyte.gz', 'rb') as lbpath:
    t_test = np.frombuffer(lbpath.read(), dtype=np.uint8, offset=8)
with gzip.open('/content/drive/MyDrive/data/t10k-images-idx3-ubyte.gz', 'rb') as imgpath:
    x_test = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(len(t_test), 784)


### DatasetとDataloader定義

In [5]:
# 前処理 -> 必要があれば編集してください
data_transform = transforms.Compose([
    transforms.ToTensor()
])

# Dataset定義 -> 必要があれば編集してください
class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train, transform):
        self.x_train = x_train
        self.t_train = t_train
        self.transform = transform

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        img = np.uint8(self.x_train[idx]).reshape(28, 28)
        img = self.transform(Image.fromarray(img, mode="zL"))
        return img, torch.tensor(self.t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test, t_test, transform):
        self.x_test = x_test
        self.t_test = t_test
        self.transform = transform

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        img = np.uint8(self.x_test[idx]).reshape(28, 28)
        img = self.transform(Image.fromarray(img, mode="L"))
        return img, torch.tensor(self.t_test[idx], dtype=torch.long)

trainval_data = train_dataset(x_train, t_train, transform=data_transform)
test_data = test_dataset(x_test, t_test, transform=data_transform)

In [5]:
# Dataloader 定義 -> 必要があれば編集してください

batch_size = 128

train_size = len(trainval_data) - val_size
train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

### 畳み込みニューラルネットワーク(CNN)の実装

- 各レイヤーの定義 -> コードを作成してください．

In [ ]:
class Conv(nn.Module):
    # WRITE ME
    

class Pooling(nn.Module):
    # WRITE ME
    

class Flatten(nn.Module):
    # WRITE ME


class Dense(nn.Module):
    # WRITE ME

- 畳み込みニューラルネットワークモデル定義 -> コードを作成してください．

In [ ]:
class ClassificationModel(nn.Module):
    # WRITE ME
    def __init__(self):
        super(ClassificationModel, self).__init__()

    def forward(self, x):
      return x

- モデルの引用

In [ ]:
# モデルの引用
rng = np.random.RandomState(1234)
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

conv_net = ClassificationModel()
conv_net.to(device)

ClassificationModel(
  (conv_net): Sequential(
    (0): Conv()
    (1): Pooling()
    (2): Conv()
    (3): Pooling()
    (4): Flatten()
    (5): Dense()
  )
)

### モデルトレーニング
- モデルトレーニングのコードを作成してください．

In [ ]:
from tqdm.notebook import tqdm
from collections import OrderedDict

n_epochs = 10
optimizer = optim.Adam(conv_net.parameters(), lr=1e-3)

for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []
    train_num = 0
    train_true_num = 0
    valid_num = 0
    valid_true_num = 0

    processbar = tqdm(total = (train_size // batch_size + 1))
    processbar.set_description("Epoch %02d" % (epoch + 1))
    conv_net.train()  # 訓練時には勾配を計算するtrainモードにする
    for x, t in dataloader_train:
        # WRITE ME 


        losses_train.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        train_num += acc.size()[0]
        train_true_num += acc.sum().item()

        processbar.set_postfix(OrderedDict(loss=loss.tolist(), acc=(acc.sum().item()/acc.size()[0])))
        processbar.update(1)

    conv_net.eval()  # 評価時には勾配を計算しないevalモードにする
    for x, t in dataloader_valid:
        # WRITE ME


        losses_valid.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        valid_num += acc.size()[0]
        valid_true_num += acc.sum().item()

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch+1,
        np.mean(losses_train),
        train_true_num/train_num,
        np.mean(losses_valid),
        valid_true_num/valid_num
    ))

  0%|          | 0/391 [00:00<?, ?it/s]

EPOCH: 1, Train [Loss: 0.629, Accuracy: 0.775], Valid [Loss: 0.471, Accuracy: 0.828]


  0%|          | 0/391 [00:00<?, ?it/s]

EPOCH: 2, Train [Loss: 0.435, Accuracy: 0.846], Valid [Loss: 0.405, Accuracy: 0.855]


  0%|          | 0/391 [00:00<?, ?it/s]

...

EPOCH: 8, Train [Loss: 0.291, Accuracy: 0.897], Valid [Loss: 0.293, Accuracy: 0.895]


  0%|          | 0/391 [00:00<?, ?it/s]

EPOCH: 9, Train [Loss: 0.279, Accuracy: 0.901], Valid [Loss: 0.301, Accuracy: 0.891]


  0%|          | 0/391 [00:00<?, ?it/s]

EPOCH: 10, Train [Loss: 0.269, Accuracy: 0.904], Valid [Loss: 0.279, Accuracy: 0.897]


### モデル分類精度テスト

In [ ]:
conv_net.eval()

t_pred = []
test_num = 0
test_true_num = 0
for x, t in dataloader_test:
    # Write Me ?
    x = x.to(device)
    y = conv_net.forward(x)
    pred = y.argmax(1) 
    acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
    test_num += acc.size()[0]
    test_true_num += acc.sum().item()

print("Accuracy on test set: {:.3f}".format(test_true_num/test_num))

Accuracy on test set: 0.892
